In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
import os
import nltk
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
#from genism.models import Word2Vec
print(os.listdir("../input"))
print(os.listdir(".."))
import spacy
#from spacy import displacy
#from topia.termextract import extract
# Any results you write to the current directory are saved as output.

In [ ]:
df2=pd.read_csv("../input/us-technology-jobs-on-dicecom/dice_com-job_us_sample.csv")
print(df2.head())
jobs=[]
for job_title  in df2.jobtitle:
    if(job_title.lower() not in jobs):
        jobs.append(job_title)
#print(jobs)
job_description=np.asarray(df2.loc[:,"jobdescription"])
print(len(job_description[0:5]))
#nlp=spacy.load('en')

def remove_whitespace_entities(doc):
    doc.ents=[x for x in doc.ents if not (x.text.isspace())]
    return doc
#nlp.add_pipe(remove_whitespace_entities,after='ner')
#article=job_description[2]
#doc=nlp(article)
#article=[x for x in nlp(article) if not x.is_stop and x.pos!='PUNCT']
#article=[x.lemma_ for x in article]

#Using Named Entity Recognition
# displacy.render(nlp(str(article)), jupyter=True, style='ent')
# nlp(str(article)).ents

#Comparing similarity of each word with a given set of words. If the similarity score is high, the word is related to technology. Hence, it should be considered.
df_languages=pd.read_excel('../input/fr-pl/languages.xlsx')
df_frameworks=pd.read_excel("../input/fr-pl/frameworks.xlsx",header=None,error_bad_lines=False,delim_whitespace=True)
experience_regex=['\d+ years','\d+ experience','']
#print(df_frameworks)
frame=[str(x).split(",")[0] for x in df_frameworks.iloc[:,0]]
print(len(df_frameworks.columns))
dictionary=list(df_languages.iloc[:,0])
dictionary.extend(frame)
print(dictionary)
dictionary=[x.lower() for x in dictionary]
extracted_jobs=dict()
for i in range(len(job_description[:10000])):
    job_id=df2.iloc[i,-1]
    job=df2.iloc[i,3]
    flag=0
    for word in job.split(" "):
        word=word.lower()
        if word in dictionary:
            flag=1
            if job_id not in extracted_jobs.keys():
                extracted_jobs[job_id]=[]
            if word not in extracted_jobs[job_id]:
                extracted_jobs[job_id].append(word)
    if(flag==0):
        print(job_id)
print(extracted_jobs)
print(len(extracted_jobs))        
            
#doc_given_text=nlp(u'computer science')
#sample_word=nlp(u'Java')
#words=[]
#for ele in doc:
#    if(doc_given_text.similarity(ele)>0.5):
#        words.append(ele.text)
#print(words)
#print(doc_given_text.similarity(sample_word))
#print(doc_given_text.similarity(doc[3]))
#print(doc[3].vector)


In [ ]:
df2=pd.read_csv("../input/us-technology-jobs-on-dicecom/dice_com-job_us_sample.csv")
print(df2.head())
jobs=[]
for job_title  in df2.jobtitle:
    if(job_title.lower() not in jobs):
        jobs.append(job_title)
#print(jobs)
job_skills=np.asarray(df2.loc[:,"skills"])
print(len(job_description[0:5]))
#nlp=spacy.load('en')

def remove_whitespace_entities(doc):
    doc.ents=[x for x in doc.ents if not (x.text.isspace())]
    return doc
#nlp.add_pipe(remove_whitespace_entities,after='ner')
#article=job_description[2]
#doc=nlp(article)
#article=[x for x in nlp(article) if not x.is_stop and x.pos!='PUNCT']
#article=[x.lemma_ for x in article]

#Using Named Entity Recognition
# displacy.render(nlp(str(article)), jupyter=True, style='ent')
# nlp(str(article)).ents

#Tokenizing words
extracted_skills=dict()
training_range=int(0.7*len(job_skills))
for i in range(training_range):
    #print(i)
    #Method 1: Manual pre-processing
    job_id=df2.iloc[i,-1]
#     job=df2.iloc[i,-2]
#     words=job_skills[i].split(",")
#     if(words[0].lower()=="(see job description)"):
#         continue
#     #print(words)
#     #print(len(words))
#     for i in range(len(words)):
#         #print(type(word))
#         chunk=words[i].split("/")
#         words.remove(words[i])
#         words.extend(chunk)
            
#     extracted_skills[job_id]=[]
#     extracted_skills[job_id].extend(words)
    #Method 2:Using NLTK
    tokenizer=nltk.tokenize.RegexpTokenizer(r'\w+')
    #print(job_skills[i])
    if(pd.isnull(job_skills[i])):
        continue
    stopwords_list=stopwords.words("english")
    tokens=re.split("|".join([","," and","/"," AND"," or"," OR",";"]),job_skills[i])
    tokens=list(set(tokens))
    #tokens=[x for x in tokens if x.isalpha()==True]
    #tokens=tokenizer.tokenize(job_skills[i])
    #print(tokens)
    #tokens=nltk.word_tokenize(job_skills[i])
    #print(stopwords)
    #stopwords_list=stopwords
    #words=[x for x in tokens if x not in stopwords_list]
    #print(tokens)
    extracted_skills[job_id]=[]
    extracted_skills[job_id].extend(tokens)
    #print(extracted_skills[job_id])
print(extracted_skills)    
    

In [ ]:
class job_postings:    
    def __init__(self,link):
        self.df2=pd.read_csv(link)
    def clean_skills(self,training_range):
        extracted_skills=dict()
        job_skills=np.asarray(self.df2.loc[:,"skills"])
        for i in range(training_range):
            #print(i)
            #Method 1: Manual pre-processing
            job_id=self.df2.iloc[i,-1]
            #Method 2:Using NLTK
            tokenizer=nltk.tokenize.RegexpTokenizer(r'\w+')
            #print(job_skills[i])
            if(pd.isnull(job_skills[i])):
                continue
            stopwords_list=stopwords.words("english")
            tokens=re.split("|".join([","," and","/"," AND"," or"," OR",";"]),job_skills[i])
            tokens=list(set(tokens))
            extracted_skills[job_id]=[]
            extracted_skills[job_id].extend(tokens)
        return extracted_skills
    def extract_skills(self,extracted_skills):
        df_languages=pd.read_excel('../input/fr-pl/languages.xlsx')
        df_frameworks=pd.read_csv("../input/new-fr/frameworks.csv")
        df_database=pd.read_csv("../input/databse/database.csv")
        df_os=pd.read_csv("../input/operating-sys/operating_systems.csv")
        df_plat=pd.read_csv("../input/platform/platforms.csv")
        frameworks=df_frameworks.iloc[:,1].tolist()
        frameworks=[x.lower().strip() for x in frameworks]
        #frameworks=[str(x).split(",")[0] for x in df_frameworks.iloc[:,1]]
        languages=list(df_languages.iloc[:,0])
        languages=[x.lower().strip() for x in languages]
        #frameworks=[x.lower().strip().split('\t')[0] for x in frameworks]
        databases=df_database.iloc[:,0].tolist()
        databases=[x.lower().strip() for x in databases]
        op_systems=df_os.iloc[:,0].tolist()
        op_systems=[x.lower().strip() for x in op_systems]
        platforms=df_plat.iloc[:,1].tolist()
        #print(platforms)
        platforms=[x.lower().strip() for x in platforms]
        #print(frameworks)
        new_extracted=dict()
        for ele in extracted_skills.keys():
            final_lang=''
            final_frame=''
            final_others=''
            final_database=''
            final_plat=''
            final_os=''
            #print(extracted_skills[ele])
            for skill in extracted_skills[ele]:
                skill_base=skill.lower().strip()
                #print(skill_base)
                if(skill_base in languages):
                    if(final_lang==''):
                        final_lang=skill_base
                    else:
                        final_lang=final_lang+","+skill_base
                elif(skill_base in frameworks):
                    if(final_frame==''):
                        final_frame=skill_base
                    else:
                        final_frame=final_frame+","+skill_base
                elif(skill_base in databases):
                    if(final_database==''):
                        final_database=skill_base
                    else:
                        final_database=final_database+","+skill_base
                elif(skill_base in op_systems):
                    if(final_os==''):
                        final_os=skill_base
                    else:
                        final_os=final_os+","+skill_base
                elif(skill_base in platforms):
                    if(final_plat==''):
                        final_plat=skill_base
                    else:
                        final_plat=final_plat+","+skill_base
                else:
                    if(final_others==''):
                        final_others=skill_base
                    else:
                        final_others=final_others+","+skill_base
            new_extracted[ele]=[final_lang,final_frame,final_database,final_os,final_plat,final_others]
        extracted_skills_df=pd.DataFrame.from_dict(new_extracted,orient='index',columns=['Language','Framework','Database','OS','Platform','Others'])
        return extracted_skills_df
    def create_job_profile(self,extracted_skills_df,domain_df):
        job_id=extracted_skills_df.index.tolist()
        languages_df=pd.DataFrame(index=job_id)
        platforms_df=pd.DataFrame(index=job_id)
        frameworks_df=pd.DataFrame(index=job_id)
        databases_df=pd.DataFrame(index=job_id)
        
        for job,lang,frame,plat,datab in list(zip(job_id,extracted_skills_df.loc[:,'Language'].tolist(),extracted_skills_df.loc[:,'Framework'].tolist(),extracted_skills_df.loc[:,'Platform'].tolist(),extracted_skills_df.loc[:,'Database'].tolist())):
            #Languages
            l=lang.split(",")
            if(lang!=np.nan or lang!=''):
                for ele in l:
                    if(ele==''):
                        continue
                    if(ele not in languages_df.columns):
                        #languages.append(ele)
                        languages_df[ele]=np.nan
                    languages_df.loc[job,ele]=1
            
            #Frameworks
            l=frame.split(",")
            if(frame!=np.nan or frame!=''):
                for ele in l:
                    if(ele==''):
                        continue
                    if(ele not in frameworks_df.columns):
                        #languages.append(ele)
                        frameworks_df[ele]=np.nan
                    frameworks_df.loc[job,ele]=1

            #Platforms
            l=plat.split(",")
            if(plat!=np.nan or plat!=''):
                for ele in l:
                    if(ele==''):
                        continue
                    if(ele not in platforms_df.columns):
                        #languages.append(ele)
                        platforms_df[ele]=np.nan
                    platforms_df.loc[job,ele]=1
            
            #Databases
            l=datab.split(",")
            if(datab!=np.nan or datab!=''):
                for ele in l:
                    if(ele==''):
                        continue
                    if(ele not in databases_df.columns):
                        #languages.append(ele)
                        databases_df[ele]=np.nan
                    databases_df.loc[job,ele]=1
        languages_df=languages_df.reindex_axis(sorted(languages_df.columns), axis=1)
        frameworks_df=frameworks_df.reindex_axis(sorted(frameworks_df.columns), axis=1)
        platforms_df=platforms_df.reindex_axis(sorted(platforms_df.columns), axis=1)
        databases_df=databases_df.reindex_axis(sorted(databases_df.columns), axis=1)
        domain_df=domain_df.reindex_axis(sorted(domain_df.columns), axis=1)

        languages_df.to_csv("languages_job_profile.csv")
        frameworks_df.to_csv("frameworks_job_profile.csv")
        platforms_df.to_csv("platforms_job_profile.csv")
        databases_df.to_csv("databases_job_profile.csv")
        domain_df.to_csv("domain_job_profile.csv")
        print(languages_df.columns)
obj=job_postings("../input/us-technology-jobs-on-dicecom/dice_com-job_us_sample.csv")
training_range=int(0.7*len(job_skills))
extracted_skills=obj.clean_skills(training_range)
extracted_skills_df=obj.extract_skills(extracted_skills)
print(extracted_skills_df)
domain_df=pd.read_csv("../input/domain-df/preprocessed_df.csv")
obj.create_job_profile(extracted_skills_df,domain_df)
#extracted_skills_df.to_csv("skill_extracted.csv")

Part A: Building the content analyzer

In [ ]:
#Identifying various categories in job postings

#Method1: Applying TF-IDF on the dataset
count=0
docs=[]
for i in range(len(job_description[:100])):
    #print(job_description[i])
    if(job_description[i]==np.nan):
        continue
    doc=[x for x in job_description[i].split(" ") if x not in stopwords_list]
    docs.append(" ".join(doc))
print(len(docs))
vectorizer=TfidfVectorizer(ngram_range=(1,2),max_df=0.6,max_features=50)
response=vectorizer.fit_transform(docs)
name_to_index=vectorizer.get_feature_names()
response=response.toarray()
scores=pd.DataFrame(data=response[:,:],index=range(len(response)),columns=name_to_index)
print(scores)
max_col_scores={}
#print(scores.iloc[0,:])
for col in range(len(scores.iloc[0,:])):
    col_score=sum(scores.iloc[:,col])
    max_col_scores[name_to_index[col]]=col_score
max_col_scores=sorted(max_col_scores.items(),reverse=True,key=lambda x:x[1])[:50]
print(max_col_scores)




In [ ]:
#Method 2: Use a separate domain field in the dataset.
#Use TF-DF on job titles
def cluster_job_titles():
    job_titles=df2.loc[:,'jobtitle'].tolist()
    #Tokenization   
    
    docs=[]
    for i in range(len(job_titles[:training_range])):
        #print(job_description[i])
        if(job_titles[i]==np.nan):
            continue
        doc=[x for x in job_description[i].split(" ") if x not in stopwords_list]
        docs.append(" ".join(doc))
    print(len(docs))
    vectorizer=TfidfVectorizer(ngram_range=(1,2),max_df=1.0,max_features=50)
    response=vectorizer.fit_transform(docs)
    model=KMeans(n_clusters=10,init='k-means++')
    model.fit(response)
    labels=model.labels_
    return labels
#name_to_index=vectorizer.get_feature_names()
#response=response.toarray()
#scores=pd.DataFrame(data=response[:,:],index=range(len(response)),columns=name_to_index)


In [ ]:
# labels=cluster_job_titles()
# cluster_mapping=dict()
# for i in range(len(labels)):
#     label=labels[i]
#     if(label not in cluster_mapping.keys()):
#         cluster_mapping[label]=[]
#     cluster_mapping[label].append(job_titles[i])
# print(cluster_mapping)
# print(len(cluster_mapping))

In [ ]:
# #Predefined categories
# #Compare similarities of word embeddings
# nlp=spacy.load('en_core_web_lg')
# # jobs=df2.loc[:,'jobtitle'].tolist()[:50]
# # jobs_descriptions=df2.loc[:,'jobdescription'].tolist()[:50]
# # for job in jobs:
# #     #print(job)
# #     doc=nlp(job)
# #     print(doc.ents)
#     #print("Entity is",ele.text,"Label",ele.label_)
# def check_threshold(threshold,ele):
#     if(ele[0]!=threshold[0][0] and abs(ele[1]-threshold[0][1])<0.03 and ele[1]>0.5):
#         return True
#     else:
#         return False
# def categorize(training_range):
#     job_id=df2.loc[:,'uniq_id'].tolist()[:training_range]
#     job_titles=df2.loc[:,'jobtitle'].tolist()[:training_range]
#     job_descriptions=df2.loc[:,'jobdescription'].tolist()[:training_range]
#     final_cat=pd.DataFrame(index=job_id)
#     #categories=['Network Engineer','Application Development','Big Data','Data Analyst','Software Developer','DevOps','Software Testing','Front End','Back End','Full Stack','Web Development','Information Security','Mobile developer','System Administrator','Business Analyst','Manager','Cloud']
#     categories=['Network Engineer','Full stack','QA/Test Developer','Enterprise application','DevOps','Mobile Developer','Back End','Database Administrator(DBA)','Front End','Game developer','System Administrator','Data Scientist','Business analyst','Sales professional','Product Manager','Information Security','Software Developer/Java Developer','Web Developer']
#     for category in categories:
#         final_cat[category]=np.nan
#     for job_t_d in list(zip(job_id,job_titles,job_descriptions)):
#         id_job=job_t_d[0]
#         job_i=job_t_d[1]
#         job_d=job_t_d[2]
#         job_title=nlp(job_i.lower())
#         job_description=nlp(job_d.lower())
#         match_cat_title=dict()
#         match_cat_description=dict()
#         for category in categories:
#             word=nlp(category.lower())
#             match_cat_title[category]=job_title.similarity(word)
#             match_cat_description[category]=job_description.similarity(word)
#         match_cat_title=sorted(match_cat_title.items(),key=lambda x:x[1],reverse=True)
#         match_cat_description=sorted(match_cat_description.items(),key=lambda x:x[1],reverse=True)
        
        
#         #a represents max
#         if(match_cat_title[0][1]>0.5 or match_cat_description[0][1]>0.5):
#             a=match_cat_title[0]
#             print(a)
#             match_cat_description=list(filter(lambda x: check_threshold(match_cat_title,x),match_cat_description))
#             if(len(match_cat_description)!=0):
#                 print(match_cat_description)
#                 print(id_job)
#                 #b=match_cat_description[0]
#                 final_cat.loc[id_job,a[0]]=1
#                 match_cat_description.extend([(match_cat_title[0][0],1)])
#                 sum_proportion=sum([x[1] for x in match_cat_description])
#                 for ele in match_cat_description:
#                     final_cat.loc[id_job,ele[0]]=ele[1]/sum_proportion
#             else:
#                 print(id_job)
#                 final_cat.loc[id_job,a[0]]=1

#         else:
#             print("not considering",job_i)
#         #print(match_cat)
#     return final_cat
# training_range=int(0.7*len(df2.loc[:,'uniq_id']))
# final_cat=categorize(training_range)

# print(final_cat)
# final_cat.to_csv("preprocessed_df.csv")

In [ ]:
#Extracting years of experience
nlp=spacy.load('en_core_web_lg')
job_description=df2.loc[:,'jobdescription'].tolist()
id_job=df2.loc[:,'uniq_id'].tolist()
experience_regex=['\d+ years \w+ $\.',r'\d+ experience']
matches=dict()
entities=dict()
for job_id,description in list(zip(id_job,job_description))[:10]:
    #l=re.findall(r"\w* experience[\S*\s*]\w*[.]",description)
    l=re.findall(r"[^.]*experience[^.]*\.",description)
    matches[job_id]=l    
    for string in matches[job_id]:
        print(string)
        doc=nlp(string)
        
        for token in doc:
            print(token.text,token.dep_,token.head.text)
print(matches)
#print(entities)

In [ ]:
#Identifying 

Part B: Profile Learner

In [ ]:
#Explicit/Implicit ways of identifying user preferences
#1) Use a like/dislike method to indicate user preference:
#Optimal method: Model based user-preference method to infer a score for each job(item) that the user has worked for in the past 